In [1]:
import pickle
import pandas as pd
import spacy
import re
from time import time
import multiprocessing

# Embedding model trainings

### Embed company descriptions

#### Import Data

In [2]:
with open("pipes/pipe4", "rb") as fp:   # Unpickling
    pipe4 = pickle.load(fp)

In [3]:
pipe4[0]

[['le',
  'fourgon',
  'delivers',
  'stored',
  'drinks',
  'home',
  'order',
  'placed',
  'lefourgon.com',
  'beers',
  'juices',
  'sodas',
  'water',
  'milk',
  'wines',
  'soups',
  'spirits',
  'co.',
  'deliver',
  'home',
  'free',
  'charge',
  'chosen',
  'niche',
  'next',
  'visit',
  'collect',
  'empty',
  'bottles',
  'return',
  'washed',
  'producer',
  'reuse',
  'zerodechet']]

In [4]:
company_df = pd.read_csv("data/company_desc_translated.csv", sep=";")[["company_name", "description_en"]]

In [5]:
company_df

,company_name,description_en
0,Le Fourgon,Le Fourgon delivers your stored drinks to your...
1,Comptoir des Vignes,Comptoir des Vignes is a brand of cellars spec...
2,Shin Sekai,Welcome to our Trustpilot page! Shin Sekai is ...
3,Nutri Naturel,"Nutri-Naturel.com, the leading online organic ..."
4,Maison Martin - Le Piment Français,Maison Martin - Le Piment Francais is the firs...
...,...,...
12991,Ljbautoparts,"Sale of auto body spare parts online: fender, ..."
12992,Aéroports de Paris,"Aeroports de Paris, with its three platforms, ..."
12993,Online SAS,"Shared hosting with unlimited traffic, domain ..."
12994,shopequitation,Online specialist in the sale of horse riding ...


### Data Cleaning

In [6]:
nlp = spacy.load("en_core_web_sm") 

In [7]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop and len(token.text) > 2]

    if len(txt) > 2:
        return ' '.join(txt)

In [8]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in company_df['description_en'])

In [9]:
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

In [10]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(11365, 1)

In [11]:
df_clean

,clean
0,fourgon deliver store drink home order place l...
1,comptoir des vigne brand cellar specialize win...
2,welcome trustpilot page shin sekai online figu...
3,nutri naturel com lead online organic grocery ...
4,maison martin piment francais brand artisanal ...
...,...
12991,sale auto body spare part online fender bumper...
12992,aeroport paris platform major connection point...
12993,share host unlimited traffic domain dedicated ...
12994,online specialist sale horse ride equipment sa...


In [12]:
df_clean.head(1).values

array([['fourgon deliver store drink home order place lefourgon com beers juice sodas water milk wine soup spirit deliver home free charge choose niche visit collect bottle return wash producer reuse zerodechet']],
      dtype=object)

In [13]:
from gensim.models.phrases import Phrases, Phraser

In [14]:
sent = [row.split() for row in df_clean['clean']]

In [15]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

In [16]:
bigram = Phraser(phrases)

In [17]:
sentences = bigram[sent]

In [18]:
sentences[0]

['fourgon',
 'deliver',
 'store',
 'drink',
 'home',
 'order',
 'place',
 'lefourgon',
 'com',
 'beers',
 'juice',
 'sodas',
 'water',
 'milk',
 'wine',
 'soup',
 'spirit',
 'deliver_home',
 'free_charge',
 'choose',
 'niche',
 'visit',
 'collect',
 'bottle',
 'return',
 'wash',
 'producer',
 'reuse',
 'zerodechet']

In [19]:
from collections import defaultdict 

In [20]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

26671

In [21]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['offer',
 'product',
 'service',
 'france',
 'find',
 'quality',
 'good',
 'brand',
 'online',
 'support']

In [22]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

16

#### Word2Vec

In [23]:
from gensim.models import Word2Vec

In [24]:
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [25]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [26]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(4257615, 11171640)

In [27]:
similar_words = w2v_model.wv.most_similar(positive=["wine"])
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

champagne: 0.879186749458313
cellar: 0.8148442506790161
winegrower: 0.8091847896575928
whiskey: 0.807194709777832
estate: 0.7873778939247131
spirit: 0.7866882681846619
tasting: 0.7598607540130615
terroir: 0.7496309280395508
rum: 0.7323971390724182
vineyard: 0.7251990437507629


#### Fasttext

In [28]:
from gensim.models import FastText

In [29]:
fasttext_model = FastText(min_count=20,
                     window=2,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [30]:
fasttext_model.build_vocab(sentences, progress_per=10000)

In [31]:
fasttext_model.train(sentences, total_examples=fasttext_model.corpus_count, epochs=30, report_delay=1)

(4255129, 11171640)

In [35]:
similar_words = fasttext_model.wv.most_similar(positive=["manga"])
for word, similarity in similar_words:
    print(f"{word}: {similarity}")

comic_strip: 0.8626441359519958
figurine: 0.8128132820129395
pop: 0.8060959577560425
geek: 0.7959259748458862
series: 0.7792560458183289
goody: 0.7690610289573669
pokemon: 0.7632246017456055
video_game: 0.7110230922698975
games: 0.7071448564529419
bookstore: 0.6758280396461487


#### GloVe